### The equations of linear elasticity

Analysis of structures is one of the major activities of modern engineering, which likely makes the PDE modeling the deformation of elastic bodies the most popular PDE in the world. It takes just one page of code to solve the equations of 2D or 3D elasticity in FEniCS, and the details follow below.

#### PDE problem

The equations governing small elastic deformations of a body $\Omega$ can be written as

\begin{align}
−\nabla\cdot \sigma &=f\ \text{in}\ \Omega, \\
\sigma &=\lambda\ tr(\epsilon)I+2\mu\epsilon,\\
\epsilon&=\frac{1}{2}\big(\nabla u+(\nabla u)^⊤\big),
\label{eq:elasticity_equations}
\end{align}

where $\sigma$ is the stress tensor, $f$ is the body force per unit volume, $\lambda$ and $\mu$ are elasticity parameters for the material in $\Omega$, $I$ is the identity tensor, $tr$ is the trace operator on a tensor, $\epsilon$ is the symmetric strain-rate tensor (symmetric gradient), and $u$ is the displacement vector field. We have here assumed isotropic elastic conditions.

We combine and to obtain

\begin{align}
\sigma &=\lambda (\nabla\cdot u)I+\mu\big(\nabla u+(\nabla u)^⊤\big).
\end{align}

Note that the equations above can easily be transformed to a single vector PDE for $u$, which is the governing PDE for the unknown $u$ (Navier’s equation). In the derivation of the variational formulation, however, it is convenient to keep the equations split as above.

#### Variational formulation

The variational formulation of the equation of linear elasticity consists of forming the inner product of the first equation and a vector test function $v\in\hat{V}$, where $\hat{V}$ is a vector-valued test function space, and integrating over the domain $\Omega$:

\begin{align}
−\int_{\Omega}(\nabla\cdot \sigma)\cdot v\ dx&=\int_{\Omega}f\cdot v\ dx\ .
\end{align}

Since $\nabla \cdot \sigma$ contains second-order derivatives of the primary unknown $u$, we integrate this term by parts:

\begin{align}
−\int_{\Omega}(\nabla\cdot \sigma)\cdot v\ dx&=\int_{\Omega}\sigma:\nabla v\ dx−\int_{\partial\Omega}(\sigma\cdot n)\cdot v\ ds\ ,
\end{align}

where the colon operator is the inner product between tensors (summed pairwise product of all elements), and $n$ is the outward unit normal at the boundary. The quantity $\sigma\cdot n$ is known as the traction or stress vector at the boundary, and is often prescribed as a boundary condition. We here assume that it is prescribed on a part $\partial\Omega_T$ of the boundary as $\sigma\cdot n=T$. On the remaining part of the boundary, we assume that the value of the displacement is given as a Dirichlet condition. We thus obtain

\begin{align}
\int_{\Omega}\sigma:\nabla v\ dx&=\int_{\Omega}f\cdot v\ dx+\int_{\partial\Omega_T}T\cdot v\ ds\ .
\end{align}

Inserting the expression for $\sigma$ into $\sigma=\lambda(\nabla\cdot u)I+\mu(\nabla u+(\nabla u)^⊤)$ gives the variational form with $u$ as unknown. Note that the boundary integral on the remaining part $\partial\Omega\setminus \partial\Omega_T$ vanishes due to the Dirichlet condition.

We can now summarize the variational formulation as: find $u\in V$ such that

\begin{align}
a(u,v)&=L(v)\quad \forall v\in\hat{V}\ ,
\end{align}
where
\begin{align}
a(u,v)&=\int_{\Omega}\sigma(u):\nabla v\ dx\ ,\\
\sigma(u)&=\lambda(\nabla\cdot u)I+\mu(\nabla u+(\nabla u)^⊤)\ ,\\
L(v)&=\int_{\Omega}f\cdot v\ dx+\int_{\partial\Omega_T}T\cdot v\ ds.
\end{align}

One can show that the inner product of a symmetric tensor $A$ and an anti-symmetric tensor $B$ vanishes. If we express $\nabla v$ as a sum of its symmetric and anti-symmetric parts, only the symmetric part will survive in the product $\sigma :\nabla v$ since $\sigma$ is a symmetric tensor. Thus replacing $\nabla u$ by the symmetric gradient $\epsilon(u)$ gives rise to the slightly different variational form

\begin{align}
a(u,v)&=\int_{\Omega}\sigma(u):\epsilon(v)\ dx,
\end{align}

where $\epsilon(v)$ is the symmetric part of $\nabla v$:
\begin{align}
\epsilon(v)&=\frac{1}{2}(\nabla v+(\nabla v)^⊤).
\end{align}

This formulation is what naturally arises from minimization of elastic potential energy and is a more popular formulation than the previous one.

#### FEniCS implementation

#### Test problem

As a test example, we will model a clamped beam deformed under its own weight in 3D. This can be modeled by setting the right-hand side body force per unit volume to $f=(0,0,−\rho g)$ with $\rho$ the density of the beam and $g$ the acceleration of gravity. The beam is box-shaped with length $L$. We set $u=u_D=(0,0,0)$ at the clamped end, $x=0$. The rest of the boundary is traction free; that is, we set $T=0$.

#### Vector function spaces

The primary unknown is now a vector field $u$ and not a scalar field, so we need to work with a vector function space:

```python
V = VectorFunctionSpace(mesh, 'P', 1)
```

With ```u = Function(V)``` we get u as a vector-valued finite element function with three components for this 3D problem.

#### Constant vectors

For the boundary condition $u=(0,0,0)$, we must set a vector value to zero, not just a scalar. Such a vector constant is specified as ```Constant((0, 0, 0))``` in FEniCS. The corresponding 2D code would use ```Constant((0, 0))```. Later in the code, we also need ```f``` as a vector and specify it as ```Constant((0, 0, rho*g))```.

#### ```nabla_grad```

The gradient and divergence operators now have a prefix ```nabla_```. This is strictly not necessary in the present problem, but recommended in general for vector PDEs arising from continuum mechanics, if you interpret $\nabla$ as a vector in the PDE notation; see the box about ```nabla_grad``` in the section [Variational formulation](https://fenicsproject.org/pub/tutorial/sphinx1/._ftut1004.html#ftut1-ns-varform).

#### Stress computation

As soon as the displacement ```u``` is computed, we can compute various stress measures. We will compute the von Mises stress defined as $σ_M=\sqrt{\frac{3}{2}s:s}$ where $s$ is the deviatoric stress tensor

\begin{align}
s&=\sigma −\frac{1}{3}\ tr(\sigma)I
\end{align}

There is a one-to-one mapping between these formulas and the FEniCS code:

```python
s = sigma(u) - (1./3)*tr(sigma(u))*Identity(d)
von_Mises = sqrt(3./2*inner(s, s))
```
The ```von_Mises``` variable is now an expression that must be projected to a finite element space before we can visualize it:

```python
V = FunctionSpace(mesh, 'P', 1)
von_Mises = project(von_Mises, V)
plot(von_Mises, title='Stress intensity')
```

#### Scaling

It is often advantageous to scale a problem as it reduces the need for setting physical parameters, and one obtains dimensionsless numbers that reflect the competition of parameters and physical effects. We develop the code for the original model with dimensions, and run the scaled problem by tweaking parameters appropriately. Scaling reduces the number of active parameters from 6 to 2 for the present application.

In Navier’s equation for $u$, arising from the equations of linear elasticity,

\begin{align}
−(\lambda+\mu)\nabla(\nabla\cdot u)−\mu \Delta u&=f,
\end{align}

we insert coordinates made dimensionless by $L$, and $\bar{u}=u/U$, which results in the dimensionless governing equation

\begin{align}
−\beta \bar{\nabla}(\bar{\nabla}\cdot \bar{u})−\bar{\Delta}\bar{u}&=\bar{f}, \quad \bar{f}=(0,0,\gamma),
\end{align}

where $β=1+\lambda/\mu$ is a dimensionless elasticity parameter and where

\begin{align}
\gamma&=\frac{\rho g L^2}{\mu U}
\end{align}

is a dimensionless variable reflecting the ratio of the load $\rho g$ and the shear stress term $\mu \Delta u\sim \mu U/L^2$ in the PDE.

One option for the scaling is to chose $U$ such that $\gamma $ is of unit size ($U=\rho gL^2/\mu$). However, in elasticity, this leads to displacements of the size of the geometry, which makes plots look very strange. We therefore want the characteristic displacement to be a small fraction of the characteristic length of the geometry. This can be achieved by choosing U equal to the maximum deflection of a clamped beam, for which there actually exists a formula: $U=\frac{3}{2}\rho gL^2\delta^2/E$, where $\delta=L/W$ is a parameter reflecting how slender the beam is, and $E$ is the modulus of elasticity. Thus, the dimensionless parameter $\delta$ is very important in the problem (as expected, since $\delta\gg 1$ is what gives beam theory!). Taking $E$ to be of the same order as $\mu$, which is the case for many materials, we realize that $\gamma\sim \delta^{−2}$ is an appropriate choice. Experimenting with the code to find a displacement that “looks right” in plots of the deformed geometry, points to $\gamma=0.4\delta^{−2}$ as our final choice of $\gamma$.

The following simulation code implements the problem with dimensions and physical parameters $\lambda$, $\mu$, $\rho$, $g$, $L$, and $W$. However, we can easily reuse this code for a scaled problem: just set $\mu=\rho=L=1$, $W$ as $W/L (δ^{−1})$, $g=\gamma$, and $\lambda=\beta$.


In [ ]:
from fenics import *
import matplotlib.pyplot as plt

# Scaled variables
L = 1; W = 0.2
mu = 1
rho = 1
delta = W/L
gamma = 0.4*delta**2
beta = 1.25
lambda_ = beta
g = gamma

# Create mesh and define function space
mesh = BoxMesh(Point(0, 0, 0), Point(L, W, W), 10, 3, 3)
V = VectorFunctionSpace(mesh, 'P', 1)

# Define boundary condition
tol = 1E-14

def clamped_boundary(x, on_boundary):
    return on_boundary and x[0] < tol

bc = DirichletBC(V, Constant((0, 0, 0)), clamped_boundary)

# Define strain and stress

def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)
    #return sym(nabla_grad(u))

def sigma(u):
    return lambda_*div(u)*Identity(d) + 2*mu*epsilon(u)


# Define variational problem
u = TrialFunction(V)
d = u.geometric_dimension()  # space dimension
v = TestFunction(V)
f = Constant((0, 0, -rho*g))
T = Constant((0, 0, 0))
a = inner(sigma(u), epsilon(v))*dx
L = dot(f, v)*dx + dot(T, v)*ds

# Compute solution (displacement)
u = Function(V)
solve(a == L, u, bc)

# Calculate stress
s = sigma(u) - (1./3)*tr(sigma(u))*Identity(d)  # deviatoric stress
von_Mises = sqrt(3./2*inner(s, s))
V = FunctionSpace(mesh, 'P', 1)
von_Mises = project(von_Mises, V)

# Compute magnitude of displacement
u_magnitude = sqrt(dot(u, u))
u_magnitude = project(u_magnitude, V)
print('min/max u:',
      u_magnitude.vector().get_local().min(),
      u_magnitude.vector().get_local().max())

# Save solution to file in VTK format
File('elastic_beam/displacement.pvd') << u
File('elastic_beam/von_mises.pvd') << von_Mises
File('elastic_beam/magnitude.pvd') << u_magnitude
